# TensorFlow model serving with Konduit Serving

This notebook illustrates a simple client-server interaction to perform inference on a TensorFlow model using the Python SDK for Konduit Serving.  

This tutorial is split into two parts: 

1. Configuration 
2. Running the server

In [1]:
from konduit import ParallelInferenceConfig, ServingConfig, TensorFlowConfig, ModelConfigType
from konduit import TensorDataTypesConfig, ModelStep, InferenceConfiguration
from konduit.server import Server
from konduit.client import Client

import numpy as np
import os 

## Configuration

Konduit Serving works by defining a series of **steps**. These include operations such as 
1. Pre- or post-processing steps
2. One or more machine learning models
3. Transforming the output in a way that can be understood by humans

If deploying your model does not require pre- nor post-processing, only one step - a machine learning model - is required. This configuration is defined using a single `ModelStep`. 

Before running this notebook, run the `build_jar.py` script and copy the JAR (`konduit.jar`) to this folder. Refer to the [Python SDK README](https://github.com/KonduitAI/konduit-serving/blob/master/python/README.md) for details. 

Start by downloading the model weights to the `data` folder. 

In [ ]:
from urllib.request import urlretrieve 
from zipfile import ZipFile
dl_path = "../data/bert/bert.zip"
if not os.path.isfile(dl_path):
    urlretrieve("https://deeplearning4jblob.blob.core.windows.net/testresources/bert_mrpc_frozen_v1.zip", 
                dl_path)
with ZipFile(dl_path, 'r') as zipObj:
    zipObj.extractall()

### Configuring `ModelStep` 

Define the TensorFlow configuration as a `TensorFlowConfig` object. 

- `tensor_data_types_config`: The TensorFlowConfig object requires a dictionary `input_data_types`. Its keys should represent column names, and the values should represent data types as strings, e.g. `"INT32"`. See [here](https://github.com/KonduitAI/konduit-serving/blob/master/konduit-serving-api/src/main/java/ai/konduit/serving/model/TensorDataType.java) for a list of supported data types. 
- `model_config_type`: This argument requires a `ModelConfigType` object. Specify `model_type` as `TENSORFLOW`, and `model_loading_path` to point to the location of TensorFlow weights saved in the PB file format.


In [ ]:
input_data_types = {'IteratorGetNext:0': 'INT32',
                    'IteratorGetNext:1': 'INT32',
                    'IteratorGetNext:4': 'INT32'}

tensorflow_config = TensorFlowConfig(
    tensor_data_types_config = TensorDataTypesConfig(
        input_data_types=input_data_types
        ),
    model_config_type = ModelConfigType(
        model_type='TENSORFLOW',
        model_loading_path=os.path.abspath('bert_mrpc_frozen.pb')
    )
)

Now that we have a `TensorFlowConfig` defined, we can define a `ModelStep`. The following parameters are specified: 
- `model_config`: pass the TensorFlowConfig object here 
- `parallel_inference_config`: specify the number of workers to run in parallel. Here, we specify `workers=1`.
- `input_names`:  names for the input data  
- `output_names`: names for the output data

In [ ]:
input_names = list(input_data_types.keys())
output_names = ["loss/Softmax"]

tf_step = ModelStep(
    model_config=tensorflow_config,
    parallel_inference_config=ParallelInferenceConfig(workers=1),
    input_names=input_names,
    output_names=output_names
)

### Configuring the server

Specify the following:
- `http_port`: select a random port.
- `input_data_format`, `output_data_format`: Specify input and output data formats as strings. 


{% hint style="info" %}
Accepted input and output data formats are as follows:

*  Input: JSON, ARROW, IMAGE, ND4J \(not yet implemented\) and NUMPY.
*  Output: NUMPY, JSON, ND4J \(not yet implemented\) and ARROW.
{% endhint %}

In [ ]:
port = np.random.randint(1000, 65535)
serving_config = ServingConfig(
    http_port=port,
    input_data_format='NUMPY',
    output_data_format='NUMPY'
)

The `ServingConfig` has to be passed to `Server` in addition to the steps as a Python list. In this case, there is a single step: `tf_step`. 

In [ ]:
server = Server(
    serving_config=serving_config,
    steps=[tf_step]
)

By default, `Server()` looks for the Konduit Serving JAR `konduit.jar` in the directory the script is run in. To change this default, use the `jar_path` argument.

The configuration is stored as a dictionary. Note that the configuration can be converted to a dictionary using the `as_dict()` method: 

In [ ]:
server.config.as_dict()

Start the server:

In [ ]:
server.start()

### Configuring the client 

To configure the client, create a Client object with the following arguments: 
- `input_data_format`: data format passed to the server for inference
- `output_data_format`: data format returned by the server endpoint 
- `return_output_data_format`: data format to be returned to the client. Note that this argument can be used to convert the output returned from the server to the client into a different format, e.g. NUMPY to JSON.


In [ ]:
client = Client(port=port)

## Running the server 

Load some sample data from NumPy files. Note that these are NumPy arrays, each with shape (4, 128): 

In [ ]:
data_input = {
    'IteratorGetNext:0': np.load('../data/bert/input-0.npy'),
    'IteratorGetNext:1': np.load('../data/bert/input-1.npy'),
    'IteratorGetNext:4': np.load('../data/bert/input-4.npy')
}

In [ ]:
predicted = client.predict(data_input)
print(predicted)

server.stop()